In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from english_words import english_words_set

from skimage import io, color, filters
from skimage.transform import resize, rotate
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot');

In [2]:
#english_words_list = list(english_words_set)
#shortlst = english_words_list[:100]

In [ ]:
#skimage reshape 
#look at model

In [9]:
shortlst = ["lions", 'tigers']

In [10]:
def make_image():
    word = np.random.choice(shortlst)
    size = np.random.uniform(20,30)
    xpos = np.random.uniform(0,1.2)
    ypos = np.random.uniform(0,1.2)
    y = word
    #print(y)
    plt.text(xpos,ypos, y, size = size)
    plt.xlim([0,2])
    plt.ylim([0,2])
    plt.axis('off')
    plt.savefig(f'data/{y}.jpg')
    plt.close()
    X = np.array(Image.open(f'data/{y}.jpg'))
    return X, y 

In [5]:
# size = np.random.uniform(5,50)
# xpos = np.random.uniform(0,1.5)
# ypos = np.random.uniform(0,1.5)
# y = "lion"
# plt.text(xpos,ypos, y, size = size)
# plt.xlim([0,2])
# plt.ylim([0,2])


In [6]:
# y = "lion"
# sizes = range(1,10)
# xpos = [0,2]
# ypos = [0,2]
# plt.text(1.2,1.5, y, size = 50)
# plt.xlim([0,2])
# plt.ylim([0,2])


In [11]:
images = []
labels = [] 
for wrd in range(1000): 
    img, lab = make_image()
    images.append(img)
    labels.append(lab)

In [8]:
# df = pd.DataFrame()
# df["Labels"] = labels
# df["Images"] = images
# df.head()

In [12]:
lb = LabelBinarizer()
new_labels = lb.fit_transform(labels)
lb.classes_
new_labels;

In [13]:
from sklearn.model_selection import train_test_split 
X = np.array(images)
y = np.array(new_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


In [14]:
def define_model(input_shape, nb_classes, dropout, nb_filters, kernel_size,  pool_size, activation='relu',optimizer='adam'):    
    model = Sequential()  # model is a linear stack of layers (don't change)

    # note: the convolutional layers and dense layers require an activation function
    # see https://keras.io/activations/
    # and https://en.wikipedia.org/wiki/Activation_function
    # options: 'linear', 'sigmoid', 'tanh', 'relu', 'softplus', 'softsign'

    model.add(Conv2D(nb_filters,
                     (kernel_size[0], kernel_size[1]),
                     padding='valid',
                     input_shape=input_shape))  # first conv. layer  KEEP
    model.add(Activation(activation))  # Activation specification necessary for Conv2D and Dense layers

    model.add(Conv2D(nb_filters,
                     (kernel_size[0], kernel_size[1]),
                     padding='valid'))  # 2nd conv. layer KEEP
    model.add(Activation(activation))

    model.add(AveragePooling2D(pool_size=pool_size))  # decreases size, helps prevent overfitting
    model.add(Dropout(dropout))  # zeros out some fraction of inputs, helps prevent overfitting

    model.add(Flatten())  # necessary to flatten before going into conventional dense layer  KEEP
    print('Model flattened out to ', model.output_shape)

    # now start a typical neural network
    model.add(Dense(32))  # (only) 32 neurons in this layer, really?   KEEP
    model.add(Activation(activation))

    model.add(Dropout(dropout))  # zeros out some fraction of inputs, helps prevent overfitting

    model.add(Dense(nb_classes))  # 10 final nodes (one for each class)  KEEP
    model.add(Activation('sigmoid'))  # softmax at end to pick between classes 0-9 KEEP

    # many optimizers available, see https://keras.io/optimizers/#usage-of-optimizers
    # suggest you KEEP loss at 'categorical_crossentropy' for this multiclass problem,
    # and KEEP metrics at 'accuracy'
    # suggest limiting optimizers to one of these: 'adam', 'adadelta', 'sgd'
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])
    return model

In [9]:
y_train.shape

(80, 3)

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 285, 429, 6)       294       
_________________________________________________________________
activation_8 (Activation)    (None, 285, 429, 6)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 282, 426, 6)       582       
_________________________________________________________________
activation_9 (Activation)    (None, 282, 426, 6)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 70, 106, 6)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 70, 106, 6)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 44520)            

In [16]:

# important inputs to the model: don't changes the ones marked KEEP in the functions above
batch_size = 16 # number of training samples used at a time to update the weights
nb_epoch = 10      # number of passes through the entire train dataset before weights "final"
nb_filters = 6    # number of convolutional filters to use
pool_size = (2,2)  # pooling decreases image size, reduces computation, adds translational invariance
kernel_size = (4, 4)  # convolutional kernel size, slides over image to learn features
dropout = 0.1
activation='relu'
optimizer='adam'
img_rows, img_cols = X_train.shape[1],X_train.shape[2]
input_shape = (img_rows,img_cols,3)
nb_classes = 1

model = define_model(input_shape, nb_classes, dropout, nb_filters, kernel_size, pool_size)

# during fit process watch train and test error simultaneously
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)


Model flattened out to  (None, 180198)
Epoch 1/10


ValueError: in user code:

    /home/weston/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/weston/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/weston/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/weston/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/weston/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/weston/anaconda3/lib/python3.7/site-packages/tensorflow/py